In [1]:
import pandas as pd

data = pd.read_csv("groceriesSynthetic.tsv", sep = "\t")
df = pd.DataFrame(data)
df_original = pd.DataFrame(data)

In [2]:
#Checking if all the values in currency are EUR
print(df["currency"].nunique() == "EUR")

False


In [3]:
rows_with_other_currencies = df[df["currency"] != "EUR"]
print(rows_with_other_currencies)

         id formattedDate               merchantName  \
804     704    06-03-2023                        NaN   
1362   1384    25-06-2023                        NaN   
1806   1381    04-04-2023                        NaN   
2530     18    16-03-2023                        NaN   
3066   1110    01-12-2022                        NaN   
...     ...           ...                        ...   
96983   477    15-04-2023                        NaN   
97047  1763    16-08-2023                        NaN   
97815  1218    05-06-2023                        NaN   
97902  1446    30-08-2023                        NaN   
99425   985    15-07-2023  TALLINN anyone Debrashire   

                                          remittanceInfo   value currency  
804    292384******9276 either JARVE SELVERI ISETEENI...  -10.74      USD  
1362   (..3895) carry any WoltItaemerenkatu somebody new    -5.7      SEK  
1806   (..6464) always sign protect everybody improve...  -22.44      SEK  
2530   (..5274) scene p

In [4]:
pip install CurrencyConverter

Note: you may need to restart the kernel to use updated packages.


In [5]:
from currency_converter import CurrencyConverter

c = CurrencyConverter()

def currency_to_EUR(value, currency):
    if currency != "EUR":
        try:
            conversion = c.convert(value, currency, "EUR")
            return round(conversion, 2)
        except Exception as e:
            print(f"Error converting {value} {currency} to EUR: {e}")
    return value

df["value"] = df.apply(lambda row: currency_to_EUR(row["value"], row["currency"]), axis=1)

In [6]:
rows_with_other_currencies = df[df["currency"] != "EUR"]
print(rows_with_other_currencies)

         id formattedDate               merchantName  \
804     704    06-03-2023                        NaN   
1362   1384    25-06-2023                        NaN   
1806   1381    04-04-2023                        NaN   
2530     18    16-03-2023                        NaN   
3066   1110    01-12-2022                        NaN   
...     ...           ...                        ...   
96983   477    15-04-2023                        NaN   
97047  1763    16-08-2023                        NaN   
97815  1218    05-06-2023                        NaN   
97902  1446    30-08-2023                        NaN   
99425   985    15-07-2023  TALLINN anyone Debrashire   

                                          remittanceInfo  value currency  
804    292384******9276 either JARVE SELVERI ISETEENI... -10.06      USD  
1362   (..3895) carry any WoltItaemerenkatu somebody new  -0.49      SEK  
1806   (..6464) always sign protect everybody improve...  -1.93      SEK  
2530   (..5274) scene prepa

In [7]:
df = df.drop(columns="currency")

In [8]:
def findMerchant(x):
    if isinstance(x, str):
        mer = ""
        li = x.split()
        i = 0
        while li[i].isupper():
            mer + " " + li[i]
        return mer

In [9]:
#finding all the merchants in the merchantName column
merchantsFromDf = df["merchantName"].apply(lambda x: x.split()[0] if isinstance(x, str) and x.split()[0].isupper() else None).dropna().unique().tolist()
print(merchantsFromDf)

['MAKSIMARK', 'OU', 'RIMI', 'SELVER', 'KAUPLUS', 'KAUBAMAJA', 'KONSUM', 'AS', 'MAXIMA', 'VALD', 'POOD', 'COOP', 'TOIDUKAUBAD', 'TALLINN', 'R-KIOSK', 'PRISMA', 'ISETEEN', 'OÜ', 'A', 'STOCKMANN', 'LIDL', 'KESKUS', 'GROSSI', 'TURU', 'ICA', 'TURG', 'ISETEENINDUS', 'MAKSEKESKUS', 'PAIDE', 'MEIE', 'KURESSAARE', 'MARKET', 'TARTU', 'WWW.RIMI.EE', 'A&O', 'R', 'VIRU', 'KARDLA', 'VILJANDI', 'PARNU', 'NARVA', 'KRISTIINE', 'JOHVI', 'BOLT', 'NARVESEN', 'KOHTLA-JARVE', 'TOIT', 'POLVA', 'MAARDU', 'SIKUPILLI', 'ROCCA', 'EEDEN', 'PUHVET', 'GET', 'BLOOM', 'APOLLO']


In [10]:
finalMerchantList = ['MAKSIMARK', 'MAKSIMARKET','RIMI','KAUBAMAJA','SELVER', 'KONSUM','MAXIMA', 'COOP','PRISMA','STOCKMANN','LIDL', 'GROSSI', 'ICA', 'MEIE TOIDUKAUBAD',
                    'A&O', 'BOLT MARKET', 'SIKUPILLI', 'BLOOM']

In [11]:
def process_merchant_name_column(row):
    if pd.isnull(row["merchantName"]):
        for merchant in finalMerchantList:
            if merchant in row["remittanceInfo"].upper():
                return merchant
    else:
        if row["merchantName"].split()[0].isupper():
            return row["merchantName"].split()[0]
        else:
            return None

In [12]:
df["merchantName"] = df.apply(process_merchant_name_column, axis=1)

In [13]:
df = df.dropna(subset=['merchantName'])

In [14]:
# Võtsin ära praegu merchantide listist mõned elemendid, mille pärast tuli pärastisel filtreerimisel
# mitu korda sama rida kuna nad sisaldasid mitut "merchantit" selle listi põhjal

elementsToRemove = ["OU", "AS", "VALD", "OÜ", "A", "R"]

for el in elementsToRemove:
    if el in merchantsFromDf:
        merchantsFromDf.remove(el)

In [15]:
df.head(20)

,id,formattedDate,merchantName,remittanceInfo,value
0,1269,24-07-2023,MAKSIMARK,567176******6366 toward information MAKSIMARKE...,-13.59
1,319,14-06-2023,LIDL,157466******5153 pay LIDL own TARTU sound behi...,-0.89
3,1340,20-02-2023,MAKSIMARK,24/04/1997 resource kaart...878232 VILJANDI MA...,-12.9
4,604,15-05-2023,OU,14/05/1977 such kaart...983921 less OUe/Tallin...,-37.32
6,400,10-03-2023,ICA,551163******9009 which ICA AB/TARTU LOUNAKESK ...,-5.1
7,1310,04-01-2023,MAXIMA,Kaart:411726******8003 human seven community a...,-17.35
8,184,07-09-2023,RIMI,576266******5911 return RIMI/TATARI RIMI ISETE...,-4.28
9,1292,09-01-2023,KONSUM,368085******4980 difficult act KONSUM ISETEENI...,5
11,535,09-05-2023,SELVER,29/05/1985 traditional kaart...609240 huge SEL...,-21.86
14,690,05-03-2023,KONSUM,146990******6058 light IHASTE KONSUM marriage ...,-1.29


In [16]:
df.shape

(75080, 5)